# **Data Streaming using PySpark**

---

**Strucured Streaming**: Structured Streaming is a scalable and fault-tolerant stream processing engine built on the Spark SQL engine. You can express your streaming computation the same way you would express a batch computation on static data. The Spark SQL engine will take care of running it incrementally and continuously and updating the final result as streaming data continues to arrive.


In [2]:
# Load Spark engine
import findspark
findspark.init()

In [3]:
import os
import sys

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [4]:
spark = SparkSession.builder.appName("StructuredStreaming_WordCount").getOrCreate()

22/03/28 14:28:55 WARN Utils: Your hostname, Predator-G3572 resolves to a loopback address: 127.0.1.1; using 172.29.43.74 instead (on interface eth0)
22/03/28 14:28:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/28 14:28:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/28 14:28:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
# Create DataFrame representing the stream of input lines
lines = spark.readStream.format("socket").option("host", "localhost").option("port", 7000).load()

22/03/28 14:28:59 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [6]:
# Split the lines into words
words = lines.select(explode(split(lines.value, " ")).alias("word"))

# Generate running word count
wordCounts = words.groupBy("word").count()

In [7]:
# Start running the query that prints the running counts to the console
query = wordCounts.writeStream.outputMode("complete").format("console").start()

22/03/28 14:29:53 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-db7c7d83-5696-4da2-b955-0b2f0a183f72. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/03/28 14:29:53 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



In [ ]:
query.awaitTermination()